# Top10 Topic Instagram Crawling

# 게시물, 댓글, 해시태그를 크롤링해봅니다
#   

- 원하든 키워드에 대한 게시물의 URL을 검색하고
- URL을 통해 게시물의 내용, 댓글, 해시태그를 크롤링 합니다

#    

![jpeg](https://i1.wp.com/kokolevel.com/wp-content/uploads/2018/10/Instagram.png?w=1620&ssl=1)
#    

In [5]:
# 키워드에 대한 url을 검색하고 url list에 저장
# 그 url list에 대해 게시물과 댓글을 검색

In [6]:
topics = ['society', 'politics', 'economic', 'foreign', 'culture',
          'entertain', 'sports', 'digital']

total_keyword_ranking10 = [['한유총', '개학연기', '유치원', '개학', '연기', '미세먼지', '사립유치원', '기자회견', '교육감', '수도권'],
 ['대통령', '독립유공자', '황교안', '해외', '후손', '문희상', '국회', 'NSC', '5당', '오찬'],
 ['판매', '은행', '부동산신탁업', '삼겹살', '신영한투대신', '3월', '예비인가', '친환경차', '신학기', '돌파'],
 ['트럼프', '볼턴', '2020', '김정은', '대통령', '빅딜', '미중', '무역협상', '문서', '중국'],
 ['미세먼지', '벚꽃', '대체로', '현재', '전국', '오후', '유치원', '수도권', '오전', '개학연기'],
 ['보스', '문제적', '이정현', '신동엽', '결혼', '샤이니', '정준호', '모범납세자', '장동민', '표창'],
 ['우승', '박성현', '감독', '서울', '스타즈', '정규리그', '승리', '시즌', '황현수', '개막전'],
 ['S10', '갤럭시', '갤럭시S10', '개통', '출시', 'KT', '시작', '5G', '함께한', '카카오']]

In [7]:
# 해시태그를 검색한 페이지에서 ID(검색에 필요한 url)을 검색하는 함수

def crawl_ID(soup):
    
    ID_list = []
    
    while len(soup) > 16000:
        ID_first_index = soup.find('shortcode')
        ID_last_index = soup.find('edge_media_to_comment')
        ID = soup[ID_first_index+12 : ID_last_index-3]
    
        # url이 아닌것은 pass
        if(len(ID)>20):
            break
    
        # 다음 url을 찾기 위해 슬라이싱
        soup = soup[ID_last_index+100:]
        # list에 추가
        ID_list.append(ID)

    # ID list 중복처리
    ID_list = set(ID_list)
    return ID_list

In [8]:
# instagram 게시물에서 내용, 댓글 크롤링

def crawl_instagram_information(id):
    
    try:
        
        # 
        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.managed_default_content_settings.images": 2}
        chrome_options.add_experimental_option("prefs", prefs)
        driver = webdriver.Chrome('/home/apostcto/ITDA/chromedriver', chrome_options=chrome_options)
        url = 'https://www.instagram.com/p/{}'.format(id)
        driver.get(url)
        
        contents_elements = driver.find_element_by_css_selector('.C7I1f, .P9YgZ').text
        comments_elements = driver.find_elements(By.CSS_SELECTOR, '.eo2As .gElp9')[1:]
        contents_first_index = contents_elements.find('\n')
        contents_elements = contents_elements[contents_first_index:]
        contents_elements = contents_elements.replace('\n', '')
        # 페이지의 댓글을 담을 list
        url_comments_list = []
        
        # 댓글 카운트
        comment_count = 0     
    
        for comment_element in comments_elements:
   
            text = comment_element.text
            comment_index = text.find('\n')
    
            comment = text[comment_index+1:]
            
            # 댓글을 모두 담아줍니다    
            url_comments_list.append(comment)
            comment_count+=1
            
                
                
            # 차단을 막기위해 sleep
        driver.close()
        time.sleep(0.1)
            
        return contents_elements, url_comments_list, comment_count
    
    # 오류 발생시
    except:
        return [], [], 0

In [10]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from joblib import Parallel, delayed
topic_index = 0

topic_list = []
keyword_list = []

keyword_content_list = []
keyword_comments_list = []
keyword_count_list = []
keyword_comments_count_list = []

start_time = time.time()

for topic in topics:
    
    print('Topic : {}\n'.format(topic))
    
    topic_comment_list = []
    
    for keyword in total_keyword_ranking10[topic_index]:
        
        content_list = []
        comment_list = []
        keyword_comment_count = 0
        
        print('{}에 대한 게시물을 검색합니다'.format(keyword))
        
        url = 'https://www.instagram.com/explore/tags/{}'.format(keyword)
        html = requests.get(url)
        soup = BeautifulSoup(html.text, 'html.parser')
        soup = str(soup)
        ID_list = crawl_ID(soup)
        
        
        parallel = Parallel(n_jobs=-1)(delayed(crawl_instagram_information)(id) for id in ID_list)
        
        # 한번 keyword 크롤리앟면 parallel에 모두 담기니
        # 각 필요한 정보를 list에 저장
        for p in parallel:
            content_list.append(p[0])
            comment_list.append(p[1])
            keyword_comment_count+=p[2]
        
        print('{}과 관련된 게시물 갯수 : {}'.format(keyword, len(ID_list)))
        print('{}과 관련된 댓글의 갯수 : {}'.format(keyword, keyword_comment_count))
        print('\n')
        
        # 각 keyword마다 카운트, 댓글 카운트
        keyword_content_list.append(content_list)
        keyword_comments_list.append(comment_list)
        keyword_count_list.append(len(ID_list))
        keyword_comments_count_list.append(keyword_comment_count)
        
        print("걸린시간 : {}분".format(round((time.time() - start_time)/60, 1)))
        print('\n')
        
    topic_index += 1

Topic : society

한유총에 대한 게시물을 검색합니다
한유총과 관련된 게시물 갯수 : 78
한유총과 관련된 댓글의 갯수 : 390


걸린시간 : 1.1분


개학연기에 대한 게시물을 검색합니다
개학연기과 관련된 게시물 갯수 : 79
개학연기과 관련된 댓글의 갯수 : 391


걸린시간 : 2.2분


유치원에 대한 게시물을 검색합니다
유치원과 관련된 게시물 갯수 : 81
유치원과 관련된 댓글의 갯수 : 192


걸린시간 : 3.4분


개학에 대한 게시물을 검색합니다
개학과 관련된 게시물 갯수 : 79
개학과 관련된 댓글의 갯수 : 187


걸린시간 : 4.5분


연기에 대한 게시물을 검색합니다
연기과 관련된 게시물 갯수 : 79
연기과 관련된 댓글의 갯수 : 373


걸린시간 : 5.6분


미세먼지에 대한 게시물을 검색합니다
미세먼지과 관련된 게시물 갯수 : 81
미세먼지과 관련된 댓글의 갯수 : 127


걸린시간 : 6.7분


사립유치원에 대한 게시물을 검색합니다
사립유치원과 관련된 게시물 갯수 : 73
사립유치원과 관련된 댓글의 갯수 : 430


걸린시간 : 7.7분


기자회견에 대한 게시물을 검색합니다
기자회견과 관련된 게시물 갯수 : 80
기자회견과 관련된 댓글의 갯수 : 310


걸린시간 : 8.7분


교육감에 대한 게시물을 검색합니다
교육감과 관련된 게시물 갯수 : 79
교육감과 관련된 댓글의 갯수 : 212


걸린시간 : 9.8분


수도권에 대한 게시물을 검색합니다
수도권과 관련된 게시물 갯수 : 0
수도권과 관련된 댓글의 갯수 : 0


걸린시간 : 9.8분


Topic : politics

대통령에 대한 게시물을 검색합니다
대통령과 관련된 게시물 갯수 : 74
대통령과 관련된 댓글의 갯수 : 289


걸린시간 : 10.9분


독립유공자에 대한 게시물을 검색합니다
독립유공자과 관련된 게시물 갯수 : 77
독립유공자과 관련된 댓글의 갯수 : 180


걸린시간 : 11.9분


황교안에 대한 게시물을 검색

In [11]:
# 1차원 리스트로 모두 변환
for i in range(80):
    keyword_comments_list[i] = sum(keyword_comments_list[i], [])

In [12]:
# dataframe을 만들기 위해 길이를 맞춰줌

topics_80 = []
keyword_80 = sum(total_keyword_ranking10, [])
keyword_comment_ratio = []

for topic in topics:
    for i in range(10):
        topics_80.append(topic)
        
company_list = ['Instagram' for _ in range(80)]
title_list = ['NaN' for _ in range(80)]

for i in range(80):
    try:
        ratio = keyword_comments_count_list[i] / keyword_count_list[i]
        keyword_comment_ratio.append(ratio)
    except:
        keyword_comment_ratio.append(0)

In [13]:
import pandas as pd

Insta_dataframe = pd.DataFrame({'Topic':topics_80,
                                'Keyword' : keyword_80,
                                'Company' : company_list,
                                'Title' : title_list,
                                'Contents' : keyword_content_list,
                                'Comments' : keyword_comments_list,
                                'KC' : keyword_count_list,
                                'KCC' : keyword_comments_count_list,
                                'KCR' : keyword_comment_ratio})

Insta_dataframe

,Topic,Keyword,Company,Title,Contents,Comments,KC,KCC,KCR
0,society,한유총,Instagram,NaN,[#내사랑#꼬맹이#둘째제대로된 사진찍기 넘힘들다몰래#포즈연습 하니?#까부리 티난다ㅡ...,"[병설은자리잇더나?병설들어가기도힘들다던데잘됫네ㅎ, 어머 안녕하세요 :-)👍, 지극히...",78,390,5.000000
1,society,개학연기,Instagram,NaN,[.저는 기냥 지방에 사는 평범한 맘인데 이번 유치원 개학연기 사태로 올린 제 피드...,"[엄니 힘내유 ㅜㅜ, 안녕하세요 피드 잘보고가요 소통해요^^, 무기한 휴학이라든데ㅋ...",79,391,4.949367
2,society,유치원,Instagram,NaN,[#아준아인#형제#유치원#같이등원#경산#코아루#신대부적#중방유치원#영원한#사이좋게형...,"[,, @_babybin_ #육아스타그램#딸스타그램#모녀스타그램#럽스타그램#육아일기...",81,192,2.370370
3,society,개학,Instagram,NaN,"[우앙아앙!!!!🥶🥶🥶, 2019.3#Youtube 동영상 업로딩했어요.#왕초보 #...","[이것조음, @dorong20 나는 당신, ., @zza_he__e_311 #ins...",79,187,2.367089
4,society,연기,Instagram,NaN,"[봄빛🌅, 🐶: 언니야 달리자고!!!! 아 폰 꺼내지말고!!!! 나 찍지말라고!!!...",[#인물 #인물사진 #인생사진 #인물스타그램 #사진스타그램 #남자모델 #모델 #모델...,79,373,4.721519
5,society,미세먼지,Instagram,NaN,[미세먼지가 연일 기승을 부리고있지만...그래도 오늘은 방긋 패션뉴스📰📰📰애매한 간...,[#korea #seoul #fashion #street #girl #samplef...,81,127,1.567901
6,society,사립유치원,Instagram,NaN,[.저는 기냥 지방에 사는 평범한 맘인데 이번 유치원 개학연기 사태로 올린 제 피드...,"[엄니 힘내유 ㅜㅜ, 안녕하세요 피드 잘보고가요 소통해요^^, 무기한 휴학이라든데ㅋ...",73,430,5.890411
7,society,기자회견,Instagram,NaN,[지후나 너 진짜 대단하다~제가 감히 좋아해도 되겠습니까🌟🌟🌟🌟🌟🌟🌟🌟🌟#박지훈#윙...,"[멋져요🙏🏻, @soojin0725 인기짱💖, ❤️☘️🤟, 😇🌻👍, #박지훈#지훈이...",80,310,3.875000
8,society,교육감,Instagram,NaN,[한달동안 안계심#골든리트리버 #개딸램 #개상전#후추는 #교육감 #한달AS#marc...,"[이뽀랑❤️, @leesiyong0416 인형 같죠 💕 새이불 꺼내면 귀신같이 올라...",79,212,2.683544
9,society,수도권,Instagram,NaN,[],[],0,0,0.000000


In [14]:
Insta_dataframe.to_csv('instagram_dataframe', index=False)